In [ ]:
import tensorflow as tf
import pandas as pd
import zipfile, os, shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm as tq
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Cek Versi Tensorflow
print(tf.__version__)

2.5.0


In [ ]:
# Download Dataset Katarak
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download jr2ngb/cataractdataset
! chmod 600 kaggle.json
! ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle
! kaggle datasets download jr2ngb/cataractdataset

100% 3.34G/3.34G [00:40<00:00, 47.9MB/s]
100% 3.34G/3.34G [00:40<00:00, 89.1MB/s]
cataractdataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Proses Ekstrak Dataset
with zipfile.ZipFile('cataractdataset.zip','r') as z:
  z.extractall('./')

os.listdir()

['.config',
 'repository',
 'cataractdataset.zip',
 'README.md',
 'dataset',
 'sample_data']

In [ ]:
# Hapus Direktori yang Tidak Digunakan
dir_path = '/content/repository'

shutil.rmtree(dir_path)

In [ ]:
base_dir = '/content/dataset'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:
# Menerapkan Fungsi Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.75 and logs.get('val_accuracy')>0.75):
            print("\nAkurasi telah mencapai lebih dari 75%, proses dihentikan!")
            self.model.stop_training = True

In [ ]:
# Proses Split Dataset
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

X = df['path']
y = df['tag']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=300)

df_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'train'})
df_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

print('train size', len(df_tr))
print('val size', len(df_val))

df_all = df_tr.append([df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')

df_all.sample(3)

train size 540
val size 61
set         tag             
train       1_normal            268
            2_cataract           92
            2_glaucoma           89
            3_retina_disease     91
validation  1_normal             32
            2_cataract            8
            2_glaucoma           12
            3_retina_disease      9
dtype: int64 



,path,tag,set
33,/content/dataset/1_normal/NL_245.png,1_normal,train
271,/content/dataset/3_retina_disease/Retina_031.png,3_retina_disease,train
61,/content/dataset/1_normal/NL_184.png,1_normal,train


In [ ]:
datasource_path = base_dir
dataset_path = "content/dataset/dataset/"

for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']            
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])  
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [ ]:
# Image Augmentation
train_dir = "content/dataset/dataset/train"
val_dir = "content/dataset/dataset/validation"

train_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
                  rescale=1./255,
                  rotation_range=40,
                  vertical_flip=True,
                  shear_range = 0.2,
                  zoom_range=0.2,
                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150,150),          
        class_mode='categorical')
 
validation_generator = val_datagen.flow_from_directory(
        val_dir, # direktori data validasi
        target_size=(150, 150), 
        class_mode='categorical')

Found 540 images belonging to 4 classes.
Found 61 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(150, 150,3), include_top=False, weights="imagenet")

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(4, activation='sigmoid'))

In [ ]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 10, epochs = 30)

Epoch 1/30
10/10 [==============================] - 175s 17s/step - loss: 4.1494 - accuracy: 0.4177 - val_loss: 4.4938 - val_accuracy: 0.1967
Epoch 2/30
10/10 [==============================] - 164s 16s/step - loss: 3.4159 - accuracy: 0.4114 - val_loss: 4.8323 - val_accuracy: 0.1967
Epoch 3/30
10/10 [==============================] - 163s 16s/step - loss: 2.8761 - accuracy: 0.4430 - val_loss: 5.3959 - val_accuracy: 0.1967
Epoch 4/30
10/10 [==============================] - 162s 16s/step - loss: 3.2780 - accuracy: 0.4146 - val_loss: 5.7596 - val_accuracy: 0.1967
Epoch 5/30
10/10 [==============================] - 163s 16s/step - loss: 2.9105 - accuracy: 0.4525 - val_loss: 6.0984 - val_accuracy: 0.1967
Epoch 6/30
10/10 [==============================] - 166s 16s/step - loss: 2.7652 - accuracy: 0.4719 - val_loss: 6.7136 - val_accuracy: 0.1967
Epoch 7/30
10/10 [==============================] - 167s 16s/step - loss: 2.5203 - accuracy: 0.4750 - val_loss: 7.0051 - val_accuracy: 0.1967
Epoch 

In [ ]:
tf.keras.models.save_model(base_model,'my_model.hdf5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
# Konversi model.
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
tflite_model = converter.convert()

with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpwkytctel/assets
